In [23]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
from jupyterthemes import jtplot
jtplot.style(theme='monokai')
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
jtplot.style(ticks=True, grid=False, figsize=(10, 10))
import datetime
from psaw import PushshiftAPI


In [80]:
api = PushshiftAPI()
features = ["title", "id", "score", "author", "created_utc", "selftext"]
subreddit = "asoiaf"
date1 = int(datetime.datetime(2015,7,16).timestamp())
date2 = int(datetime.datetime(2019,4,14).timestamp())
gen = api.search_submissions(subreddit = subreddit, after = date1, before = date2, q='', filter = features, limit =3 * 10**5)

In [81]:
results = list(gen)

/home/oskar/.local/lib/python3.9/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/home/oskar/.local/lib/python3.9/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [82]:

ids, title, score, author, created_utc, selftext = [], [], [], [], [], []
for i in range(len(results)):
    try:
        selftext.append(results[i].d_['selftext'])
        ids.append(i)
        title.append(results[i].d_['title'])
        score.append(results[i].d_['score'])
        created_utc.append(results[i].d_['created_utc'])
        author.append(results[i].d_['author'])
    except KeyError:
        pass
        

data = pd.DataFrame(
{
    "id" : ids,
    "title" : title,
    "score" : score,
    "created" : created_utc,
    "author" : author,
    "selftext" : selftext
}, index = None)

In [79]:
data
# data.to_csv(index=False)

,id,title,score,created,author,selftext
0,0,How Game of Thrones Lost Interest in Tyrion La...,1,1555192642,Euphoric_Translator,
1,1,(Spoilers Extended) The meeting that I'm waiti...,376,1555191033,JRR_STARK,
2,2,Unborn Lannister,1,1555191006,kcconk,[removed]
3,3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\n\nAr...
4,4,(Spoilers extended) For all those that love to...,1,1555188219,KingPoTheThird,
...,...,...,...,...,...,...
40658,41776,(Spoilers Extended) Highlighting the Shows Cas...,3,1500159470,TheCognacConnoisseur,After my Pre-Season 7 rerun through Game of Th...
40659,41777,(Spoilers Extended) Jon/Arya wolf analogy?,10,1500158083,Mrnoroboto,As listening to the audio books for a second t...
40660,41778,(Spoilers Extended) Night's Watch: The Rigged ...,31,1500157486,LeftWingScot,
40661,41779,John/Arya wolf analysis?,1,1500156395,[deleted],[removed]
